# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4880, uptime 0:00:31
mini_beamline                    RUNNING   pid 4864, uptime 0:00:33
random_walk                      RUNNING   pid 4881, uptime 0:00:31
random_walk_horiz                RUNNING   pid 4866, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4867, uptime 0:00:33
simple                           RUNNING   pid 4868, uptime 0:00:33
thermo_sim                       RUNNING   pid 4869, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4870, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-28 20:05:18
Persistent Unique Scan ID: '4362dc1e-a66b-4899-af9c-7ded19bd7f3f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:05:18.4 |     -1.000 |     101335 |


|         2 | 20:05:18.6 |     -0.500 |     101439 |
|         3 | 20:05:18.6 |      0.000 |     101244 |
|         4 | 20:05:18.6 |      0.500 |     100807 |
|         5 | 20:05:18.6 |      1.000 |      99140 |


+-----------+------------+------------+------------+
generator scan ['4362dc1e'] (scan num: 3)





('4362dc1e-a66b-4899-af9c-7ded19bd7f3f',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/4362dc1e-a66b-4899-af9c-7ded19bd7f3f-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-28 20:05:18
Persistent Unique Scan ID: '76150ff2-40bf-4b54-be07-d2f709a347ed'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:05:19.0 |          -1 | 1089682453 |


|         2 | 20:05:19.2 |          -0 | 1079780801 |


|         3 | 20:05:19.3 |           0 | 1068316614 |
|         4 | 20:05:19.4 |           0 | 1063413570 |


|         5 | 20:05:19.5 |           1 | 1059580021 |


+-----------+------------+-------------+------------+
generator scan ['76150ff2'] (scan num: 4)





('76150ff2-40bf-4b54-be07-d2f709a347ed',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-28 20:05:19
Persistent Unique Scan ID: 'd5903290-bbea-427d-a9ca-d6f43641c3e9'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:05:20.0 | 1053340471 |


|         2 | 20:05:20.9 | 1097011157 |


|         3 | 20:05:21.9 | 1131525489 |


...

|        28 | 20:05:47.0 | 1085699614 |


|        29 | 20:05:48.0 | 1053899230 |


|        30 | 20:05:49.0 | 1099990300 |


+-----------+------------+------------+
generator count ['d5903290'] (scan num: 5)





('d5903290-bbea-427d-a9ca-d6f43641c3e9',)